In [1]:
!pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
import logging
import os
import random
import re
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from datasets import load_dataset, Dataset
from sklearn.metrics import f1_score
import numpy as np
from torch import nn
import glob
import shutil
from transformers import AutoModelForMaskedLM , AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
import transformers
from transformers import (
    Trainer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback,

)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import numpy as np
from torch import nn
from transformers.file_utils import ModelOutput
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import pandas as pd
from ast import literal_eval
from scipy.special import expit
import torch
from transformers import AutoModelForQuestionAnswering
import evaluate
from tqdm import tqdm
import collections
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import default_data_collator


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
max_length = 512
stride = 128
n_best = 20
max_answer_length = 30
lr = 2e-5
EPOCHS = 2
batch_size = 4

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
device

device(type='cpu')

In [5]:

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [6]:
metric = evaluate.load("squad")

In [7]:

metric2 = evaluate.load("cuad")

In [8]:

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)



In [9]:
def convert_predictions_to_predictions2(predictions):
    predictions2 = []
    for prediction in predictions:
        # Create a new entry for predictions2
        new_entry = {
            'id': prediction['id'],  # Copy the 'id' from the original prediction
            'prediction_text': [prediction['prediction_text']]  # Wrap the single prediction text in a list
        }
        predictions2.append(new_entry)
    return predictions2


def convert_references_to_references2(references):
    references2 = []
    for reference in references:
        # Create a new entry for references2
        new_entry = {
            'id': reference['id'],  # Copy the 'id' from the original reference
            'answers': {
                'text': reference['answers']['text'],  # Keep the text as is, assuming it's already a list
                'answer_start': reference['answers']['answer_start']  # Keep answer_start as is, assuming it's already a list
            }
        }
        references2.append(new_entry)
    return references2



def compute_metrics2(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]

    predicted_answers = convert_predictions_to_predictions2(predicted_answers)
    theoretical_answers = convert_references_to_references2(theoretical_answers)

    return metric2.compute(predictions=predicted_answers, references=theoretical_answers)



In [10]:


def model_fit(model,train_dataloader,
              eval_dataloader,validation_dataset,
              validation_dataset_,
              optimizer, lr_scheduler,
              num_train_epochs = 5 ):

    epoch_num = []
    epoch_train_loss = []
    epoch_val_loss = []
    epoch_em = []
    epoch_f1 = []
    epoch_aupr = []
    epoch_p_80_r = []
    epcoh_p_90_r = []


    for epoch in range(num_train_epochs):
        # Training
        model.train()
        train_epoch_loss = 0


        for step, batch in tqdm(enumerate(train_dataloader)):

            batch = {key: value.to(device) for key, value in batch.items()}

            optimizer.zero_grad()

            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            scheduler.step()

            train_epoch_loss += loss.item()



        train_avg_loss = train_epoch_loss / len(train_dataloader)




        model.eval()
        val_epoch_loss = 0
        start_logits = []
        end_logits = []

        for batch in tqdm(eval_dataloader):
            with torch.no_grad():

                batch = {key: value.to(device) for key, value in batch.items()}

                outputs = model(**batch)

                val_epoch_loss += loss.item()


                start_logits.append(outputs.start_logits.cpu().detach().numpy())
                end_logits.append(outputs.end_logits.cpu().detach().numpy())


        val_avg_loss = val_epoch_loss / len(eval_dataloader)
        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        val_metrics = compute_metrics2(start_logits, end_logits, validation_dataset,validation_dataset_ )

        print()
        print("*"*100)
        print()
        print(f"Epoch {epoch + 1} -Train Average Loss: {train_avg_loss:.4f}")
        print(f"Eval Average Loss: {val_avg_loss:.4f}")
        print(val_metrics)
        print()
        print("*"*100)
        print()

        epoch_num.append(epoch + 1)
        epoch_train_loss.append(train_avg_loss)
        epoch_val_loss.append(val_avg_loss)
        epoch_em.append(val_metrics["exact_match"])
        epoch_f1.append(val_metrics["f1"])
        epoch_aupr.append(val_metrics["aupr"])
        epoch_p_80_r.append(val_metrics["prec_at_80_recall"])
        epcoh_p_90_r.append(val_metrics["prec_at_90_recall"])


    df = {
        "epoch" : epoch_num ,
        "train_loss" : epoch_train_loss,
        "val_loss" : epoch_val_loss,
        "epoch_em" : epoch_em,
        "epoch_f1" : epoch_f1,
        "epoch_aupr" : epoch_aupr,
        "epoch_p_80_r" :  epoch_p_80_r,
        "epcoh_p_90_r" : epcoh_p_90_r


    }

    df = pd.DataFrame(df)

    return df




In [11]:
train_dataset_ = load_dataset("squad", split = "train[:1000]")
validation_dataset_ = load_dataset("squad", split = "validation[:200]")


train_dataset = train_dataset_.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_dataset_.column_names,
)

validation_dataset = validation_dataset_.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=validation_dataset_.column_names,
)

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
train_dataset_

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})

In [13]:

train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1005
})

In [ ]:
train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=batch_size
)

In [ ]:
len(train_dataloader)

252

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.01)

steps_per_epoch = len(train_dataloader)

total_training_steps = steps_per_epoch * EPOCHS

warmup_steps = int(0.05 * total_training_steps)


scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

In [ ]:
df = model_fit(model,train_dataloader,
              eval_dataloader,validation_dataset,
              validation_dataset_,
              optimizer, scheduler,
              num_train_epochs = 5 )

In [ ]:
del model

In [ ]:
df

,epoch,train_loss,val_loss,epoch_em,epoch_f1,epoch_aupr,epoch_p_80_r,epcoh_p_90_r
0,1,3.775682,3.733918,0.0,0.0,0.0,0,0
1,2,3.398134,3.528391,0.0,0.0,0.0,0,0
2,3,3.329500,3.503206,0.0,0.0,0.0,0,0
3,4,3.355650,3.442072,0.0,0.0,0.0,0,0
4,5,3.354671,3.514510,0.0,0.0,0.0,0,0
